Worked on by:

Clean the data

This is very important on both datasets as you don't control the field-names yourself.

    Rename columns to remove spaces from the name and make them all lowercase.
    Look for the Na-values. Fix them if possible, using a couple of techniques before deciding which is best.
    Look for the Outliers. Fix them if needed, using a couple of techniques before deciding which is best.
    Look for (ordered) categoricals and label them as such.

Exploratory data analysis

    Show the outliers and try to explain them.
    Draw some graphs about features you find interesting.
    Look for correlations in your data.

Repeat

Both phases (cleaning and exploring) aren't two different steps in a process, but rather one large fuzzy mess of code. Something like:

    I'll give all the Na's the mean value.
    My graphs look strange, what is this peak doing at the mean?
    Is there a correlation between the value that has a lot of Na's and another column?
    Let's use the other column to fill in the Na's!

In the upcoming steps of this project you'll put all the really good code together to make one final import, but for now poke the data and see what happens.
Final loading and preparing

Once you've cleaning and exploring and cleaning and exploring you'll lose track of the necessary steps and the optional steps. It's also possible that you did some cleaning that wasn't needed (e.g predict a field that should have been left empty).

So end up your cleaning by creating a notebook that contains all the code to go from rough data to prepared data without any of the graphs.

This has to be done for all both datasets.

# Clean

In [ ]:
!pip install pandas matplotlib numpy openpyxl seaborn scikit-learn

In [1]:
import pandas as pd

def parse_custom_date(date_str):
    date_str = str(date_str).strip()  # Convert to string and strip spaces
    
    # Handle empty or nan-like values
    if date_str in ['nan', 'NaN', '', None]:
        return pd.NaT
    
    # Length checks (you can adjust if needed)
    length = len(date_str)
    
    try:
        if length == 10 and '-' in date_str:  # Format YYYY-MM-DD, e.g. 2001-01-01 or 2025-01-01
            return pd.to_datetime(date_str, format='%Y-%m-%d')
        elif length == 11 and '-' in date_str:  # Format DD-MMM-YYYY e.g. 01-JAN-2019
            return pd.to_datetime(date_str, format='%d-%b-%Y')
        elif length == 9 and '-' in date_str:  # Format DD-Mmm-YY e.g. 01-Jan-23
            return pd.to_datetime(date_str, format='%d-%b-%y')
        else:
            # Fallback to pandas default parsing
            return pd.to_datetime(date_str, errors='coerce')
    except Exception:
        return pd.NaT


In [2]:
import os

path = './demanddata/'
files = [f for f in os.listdir(path) if f.endswith('.csv')]

dfs = []

for file in files:
    temp_df = pd.read_csv(os.path.join(path, file))
    temp_df['SETTLEMENT_DATE'] = temp_df['SETTLEMENT_DATE'].apply(parse_custom_date)
    dfs.append(temp_df)

df = pd.concat(dfs, ignore_index=True)

print(f"Total rows combined: {len(df)}")
print(f"Missing dates after parsing: {df['SETTLEMENT_DATE'].isna().sum()}")

print(len(df))


Total rows combined: 434014
Missing dates after parsing: 0
434014


In [3]:
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

In [4]:
# Ensure date is datetime
df['settlement_date'] = pd.to_datetime(df['settlement_date'])

# Fill generation/flow columns missing values with zero
cols_fill_zero = [
    'embedded_wind_generation', 'embedded_wind_capacity',
    'embedded_solar_generation', 'embedded_solar_capacity',
    'ifa2_flow', 'britned_flow', 'east_west_flow',
    'nemo_flow', 'nsl_flow', 'eleclink_flow',
    'viking_flow', 'greenlink_flow','moyle_flow'
]
df[cols_fill_zero] = df[cols_fill_zero].fillna(0)

In [5]:
df = df.drop(['scottish_transfer','nsl_flow', 'eleclink_flow','greenlink_flow','viking_flow','east_west_flow','nemo_flow','ifa2_flow'], axis=1)

In [6]:
df = df[df['settlement_date'] >= '2006-01-01'].reset_index(drop=True)

In [7]:
df['tsd'] = df['tsd'].fillna(method='ffill')

C:\Users\thoma\AppData\Local\Temp\ipykernel_14644\3494621182.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['tsd'] = df['tsd'].fillna(method='ffill')


In [8]:
df = df[df['settlement_period'] <= 48]


In [9]:
import numpy as np
df.loc[df['tsd'] == 0, 'tsd'] = np.nan

df['tsd'] = df['tsd'].interpolate()

In [10]:
df.head()

,settlement_date,settlement_period,nd,tsd,england_wales_demand,embedded_wind_generation,embedded_wind_capacity,embedded_solar_generation,embedded_solar_capacity,non_bm_stor,pump_storage_pumping,ifa_flow,britned_flow,moyle_flow
0,2006-01-01,1,38596,39660.0,34982,0.0,0.0,0.0,0.0,0,295,1997,0.0,-169.0
1,2006-01-01,2,38829,39897.0,35312,0.0,0.0,0.0,0.0,0,299,1997,0.0,-169.0
2,2006-01-01,3,38456,39599.0,35018,0.0,0.0,0.0,0.0,0,374,1998,0.0,-169.0
3,2006-01-01,4,37401,38823.0,34054,0.0,0.0,0.0,0.0,0,653,1998,0.0,-169.0
4,2006-01-01,5,36586,37937.0,33297,0.0,0.0,0.0,0.0,0,582,1998,0.0,-169.0


In [12]:
df.to_pickle('cleaned_data.pkl')